## Create feature matrix from predictor csv files

In [1]:
import pandas as pd
import json
import re

### 1. Define raw data

In [2]:
# Path where raw data files are stored
raw_file_path = "../data/raw/"

# File names of raw data csv files to use in feature matrix
file_names = {
    "sentinel2A": "sent2_gs17to21_median_and_maxndvi.csv",
    "landsat7": "land7_gs05to14_median_and_maxndvi.csv",
    "baseline_categorical": "baseline_predictors_cat.csv",
    "baseline_numeric": "baseline_predictors_num.csv"
}

# List to store all created pd.DataFrames for merging
list_df = list()

In [3]:
with open('../data/dict/spectral_indices.json', encoding="utf-8") as json_file:
    indices_dict = json.load(json_file)

----
### 2. Load and preprocess raw data
#### 2.1 Sentinel-2A

In [4]:
# Read in Sentinel-2 data
df_sentinel = pd.read_csv(raw_file_path + file_names.get("sentinel2A")).round(decimals=5)

In [5]:
print(df_sentinel.shape)
df_sentinel.head()

(22736, 26)


,B11_median_comp,B12_median_comp,B1_median_comp,B2_median_comp,B3_median_comp,B4_median_comp,B5_median_comp,B6_median_comp,B7_median_comp,B8A_median_comp,...,GNDVI_median_comp,NDMI_greenest_pixel,NDMI_median_comp,NDVI_greenest_pixel,NDVI_median_comp,POINT_X,POINT_Y,SAVI_greenest_pixel,SAVI_median_comp,layer
0,0.2805,0.16115,0.03895,0.04585,0.06840,0.07295,0.12890,0.2231,0.25690,0.29930,...,0.58204,0.12542,0.01721,0.70803,0.54035,270096.62361,6.948292e+06,0.44184,0.30775,1ab_p
1,0.2753,0.16345,0.03380,0.05155,0.06810,0.07275,0.10480,0.1946,0.22915,0.26740,...,0.46627,0.09299,-0.03565,0.71962,0.39405,270602.65752,6.948119e+06,0.40020,0.18581,1ab_p
2,0.2025,0.11965,0.16730,0.07245,0.11145,0.11680,0.14775,0.1815,0.20835,0.24420,...,0.34606,-0.01083,0.06254,0.65992,0.31601,269874.74220,6.947897e+06,0.36472,0.20166,1ab_p
3,0.2293,0.14980,0.13685,0.25335,0.27277,0.28355,0.30945,0.3531,0.35350,0.34683,...,0.16854,-0.09972,0.14742,0.54604,0.16876,306729.77724,6.930090e+06,0.27127,0.14876,1ab_p
4,0.2815,0.15560,0.08470,0.04525,0.07025,0.07115,0.14630,0.3057,0.33605,0.37300,...,0.65050,0.13226,0.12306,0.76043,0.65026,179272.43710,6.912279e+06,0.54081,0.42310,1ab_p


In [6]:
# Drop band reflectance
band_regex = re.compile('B[\d]{1,2}.*', re.IGNORECASE)
df_sentinel = df_sentinel.drop(columns=[x for x in df_sentinel.columns if re.match(band_regex, x)])
df_sentinel.columns

Index(['EVI_greenest_pixel', 'EVI_median_comp', 'FLATE_NR',
       'GNDVI_greenest_pixel', 'GNDVI_median_comp', 'NDMI_greenest_pixel',
       'NDMI_median_comp', 'NDVI_greenest_pixel', 'NDVI_median_comp',
       'POINT_X', 'POINT_Y', 'SAVI_greenest_pixel', 'SAVI_median_comp',
       'layer'],
      dtype='object')

##### Rename columns to reflect data source

In [7]:
spectral_index_regex_list = [
    re.compile(key+'.*', re.IGNORECASE) for key in indices_dict.keys()
]

for idx, col_name in enumerate(df_sentinel.columns):
    if any([bool(re.match(x, col_name)) for x in spectral_index_regex_list]):
        df_sentinel.rename(
            columns={df_sentinel.columns[idx]: col_name + '_sent2'},
            inplace=True
        )
        
print(df_sentinel.columns)

Index(['EVI_greenest_pixel_sent2', 'EVI_median_comp_sent2', 'FLATE_NR',
       'GNDVI_greenest_pixel_sent2', 'GNDVI_median_comp_sent2',
       'NDMI_greenest_pixel_sent2', 'NDMI_median_comp_sent2',
       'NDVI_greenest_pixel_sent2', 'NDVI_median_comp_sent2', 'POINT_X',
       'POINT_Y', 'SAVI_greenest_pixel_sent2', 'SAVI_median_comp_sent2',
       'layer'],
      dtype='object')


In [8]:
# Append to list
list_df.append(df_sentinel)

-------------
#### 2.2 Landsat 7

In [9]:
### Read in Landsat-7 data
df_landsat = pd.read_csv(raw_file_path + file_names.get("landsat7")).round(decimals=5)

In [10]:
df_landsat.head()

,B1_median_comp,B2_median_comp,B3_median_comp,B4_median_comp,B5_median_comp,B6_median_comp,B7_median_comp,EVI_greenest_pixel,EVI_median_comp,FLATE_NR,...,GNDVI_median_comp,NDMI_greenest_pixel,NDMI_median_comp,NDVI_greenest_pixel,NDVI_median_comp,POINT_X,POINT_Y,SAVI_greenest_pixel,SAVI_median_comp,layer
0,2.0000,2.0000,2.0000,0.6845,0.28790,291.30,0.14650,0.43884,2.50000,2028,...,-0.49004,0.07282,0.40786,0.62089,-0.49004,270096.62361,6.948292e+06,0.40048,-0.61964,1ab_p
1,2.0000,2.0000,2.0000,0.7734,0.25155,290.45,0.13335,0.42616,2.50000,2028,...,-0.44227,0.08900,0.50915,0.63735,-0.44227,270602.65752,6.948119e+06,0.40488,-0.56208,1ab_p
2,2.0000,2.0000,2.0000,0.8127,0.23320,291.30,0.12640,0.35847,2.50000,2028,...,-0.42212,0.06869,0.55407,0.50337,-0.42212,269874.74220,6.947897e+06,0.31402,-0.53761,1ab_p
3,2.0000,2.0000,2.0000,0.5182,0.32390,292.90,0.20980,0.26667,2.50000,2227,...,-0.58844,-0.05354,0.23073,0.48459,-0.58844,306729.77724,6.930090e+06,0.24842,-0.73643,1ab_p
4,0.4278,0.4075,0.3767,0.4398,0.24340,293.90,0.13370,0.45076,0.32096,1526,...,0.03812,0.11054,0.28747,0.63519,0.07728,179272.43710,6.912279e+06,0.40628,0.07190,1ab_p


In [11]:
# Drop band reflectance
band_regex = re.compile('B[\d]{1,2}.*', re.IGNORECASE)
df_landsat = df_landsat.drop(columns=[x for x in df_landsat.columns if re.match(band_regex, x)])
df_landsat.columns

Index(['EVI_greenest_pixel', 'EVI_median_comp', 'FLATE_NR',
       'GNDVI_greenest_pixel', 'GNDVI_median_comp', 'NDMI_greenest_pixel',
       'NDMI_median_comp', 'NDVI_greenest_pixel', 'NDVI_median_comp',
       'POINT_X', 'POINT_Y', 'SAVI_greenest_pixel', 'SAVI_median_comp',
       'layer'],
      dtype='object')

In [12]:
spectral_index_regex_list = [
    re.compile(key+'.*', re.IGNORECASE) for key in indices_dict.keys()
]

for idx, col_name in enumerate(df_landsat.columns):
    if any([bool(re.match(x, col_name)) for x in spectral_index_regex_list]):
        df_landsat.rename(
            columns={df_landsat.columns[idx]: col_name+'_land7'},
            inplace=True
        )
        
print(df_landsat.columns)

Index(['EVI_greenest_pixel_land7', 'EVI_median_comp_land7', 'FLATE_NR',
       'GNDVI_greenest_pixel_land7', 'GNDVI_median_comp_land7',
       'NDMI_greenest_pixel_land7', 'NDMI_median_comp_land7',
       'NDVI_greenest_pixel_land7', 'NDVI_median_comp_land7', 'POINT_X',
       'POINT_Y', 'SAVI_greenest_pixel_land7', 'SAVI_median_comp_land7',
       'layer'],
      dtype='object')


In [13]:
### Append to list
list_df.append(df_landsat)

-----------
#### 2.3 Horvath et al. (2019)

In [14]:
### Read in Sentinel-2 data
df_baseline_cat = pd.read_csv(raw_file_path + file_names.get("baseline_categorical")).round(decimals=5)
df_baseline_num = pd.read_csv(raw_file_path + file_names.get("baseline_numeric")).round(decimals=5)

In [15]:
df_baseline_cat.head()

,FLATE_NR,POINT_X,POINT_Y,ar50_artype,ar50_skogbon,ar50_treslag,ar50_veg,corine_lc_2012,geo_berggrunn,geo_grunnvann,geo_infiltr_evne,geo_losmasse,geo_norge123,geology_norge,layer
0,2028,270096.62361,6.948292e+06,50,98,39,55,322,9,1,1,3,2,2,1ab_p
1,2028,270602.65752,6.948119e+06,50,98,39,54,333,9,1,1,3,2,2,1ab_p
2,2028,269874.74220,6.947897e+06,50,98,39,55,322,9,1,1,3,2,2,1ab_p
3,2227,306729.77724,6.930090e+06,50,98,39,52,333,12,1,4,7,1,1,1ab_p
4,1526,179272.43710,6.912279e+06,50,98,39,54,333,9,1,4,7,2,2,1ab_p


In [16]:
df_baseline_cat.columns

Index(['FLATE_NR', 'POINT_X', 'POINT_Y', 'ar50_artype', 'ar50_skogbon',
       'ar50_treslag', 'ar50_veg', 'corine_lc_2012', 'geo_berggrunn',
       'geo_grunnvann', 'geo_infiltr_evne', 'geo_losmasse', 'geo_norge123',
       'geology_norge', 'layer'],
      dtype='object')

In [17]:
df_baseline_num.head()

,FLATE_NR,POINT_X,POINT_Y,aspect,bioclim_1,bioclim_10,bioclim_11,bioclim_12,bioclim_15,bioclim_17,...,tmax_6,tmax_8,tmax_9,tmin_5,tmin_9,topographic_wetness_index,total_insolation,valley_depth,vertical_distance_to_channel_network,visible_sky
0,2028,270096.62361,6.948292e+06,2.6779,-1.21420,9.06245,-12.06838,669.31641,41.22634,102.9096,...,19.2190,17.1278,12.4009,-5.3641,-3.4312,8.9934,1320.79541,30.04170,176.19540,95.5727
1,2028,270602.65752,6.948119e+06,3.3684,-1.18491,9.11867,-12.09082,667.57703,41.37002,102.2981,...,19.2427,17.1531,12.4217,-5.2999,-3.3327,8.6541,1286.72034,9.76290,196.54530,97.7823
2,2028,269874.74220,6.947897e+06,2.1871,-1.10754,9.20473,-12.01817,666.53308,41.59916,101.7867,...,19.2683,17.1794,12.5175,-5.2076,-3.1741,8.0271,1292.41919,42.75130,163.54240,94.7781
3,2227,306729.77724,6.930090e+06,0.4182,-2.15948,9.10083,-13.84702,741.49127,33.48714,116.8842,...,20.6949,17.3567,11.6246,-6.2461,-3.5825,6.9869,1097.58044,6.29760,385.99399,97.2635
4,1526,179272.43710,6.912279e+06,0.7977,-1.27174,7.96582,-9.89952,535.50842,33.20507,83.9704,...,16.0769,15.9141,12.0073,-6.1475,-3.2039,8.9052,952.49194,344.36292,218.85001,84.4186


In [18]:
df_baseline_num.columns

Index(['FLATE_NR', 'POINT_X', 'POINT_Y', 'aspect', 'bioclim_1', 'bioclim_10',
       'bioclim_11', 'bioclim_12', 'bioclim_15', 'bioclim_17', 'bioclim_18',
       'bioclim_3', 'bioclim_5', 'bioclim_6', 'bioclim_7', 'bioclim_8',
       'bioclim_9', 'dem100', 'growing_season_length', 'layer', 'precip_5',
       'precip_6', 'proxy_allrivers', 'proxy_allwater', 'proxy_coast',
       'proxy_lakes', 'sca_2', 'sca_7', 'sca_8', 'sca_9', 'slope', 'swe_10',
       'swe_4', 'swe_8', 'terrain_ruggedness_index', 'tmax_1', 'tmax_10',
       'tmax_2', 'tmax_5', 'tmax_6', 'tmax_8', 'tmax_9', 'tmin_5', 'tmin_9',
       'topographic_wetness_index', 'total_insolation', 'valley_depth',
       'vertical_distance_to_channel_network', 'visible_sky'],
      dtype='object')

In [19]:
### Append to list
list_df.append(df_baseline_cat)
list_df.append(df_baseline_num)

----
### 3. Merge data frames

In [20]:
# Variables to use for inner merging
merge_vars = ('POINT_X', 'POINT_Y', 'FLATE_NR', 'layer')

# Do all DataFrames have the required merging column names?
dfs_have_all_keys = all(
    [(x in list_df[idx].columns) for x in merge_vars for idx in range(len(list_df))]
)

if dfs_have_all_keys and (len(list_df) > 1):
    
    # Loop through list of dataframes and merge
    for idx in range(len(list_df) - 1):
            
        if idx == 0:
            df_merged = pd.merge(
                left=list_df[idx],
                right=list_df[idx+1],
                how='inner',
                on=merge_vars
            )
        else:
            df_merged = pd.merge(
                left=df_merged,
                right=list_df[idx+1],
                how='inner',
                on=merge_vars
            )

else:
    raise ValueError(
        f"Check the data, one or more DataFrames seem to be missing the merging keys: {merge_vars}"
    )

In [21]:
df_merged.head()

,EVI_greenest_pixel_sent2,EVI_median_comp_sent2,FLATE_NR,GNDVI_greenest_pixel_sent2,GNDVI_median_comp_sent2,NDMI_greenest_pixel_sent2,NDMI_median_comp_sent2,NDVI_greenest_pixel_sent2,NDVI_median_comp_sent2,POINT_X,...,tmax_6,tmax_8,tmax_9,tmin_5,tmin_9,topographic_wetness_index,total_insolation,valley_depth,vertical_distance_to_channel_network,visible_sky
0,0.44837,0.31249,2028,0.69518,0.58204,0.12542,0.01721,0.70803,0.54035,270096.62361,...,19.2190,17.1278,12.4009,-5.3641,-3.4312,8.9934,1320.79541,30.04170,176.19540,95.5727
1,0.40884,0.21415,2028,0.67526,0.46627,0.09299,-0.03565,0.71962,0.39405,270602.65752,...,19.2427,17.1531,12.4217,-5.2999,-3.3327,8.6541,1286.72034,9.76290,196.54530,97.7823
2,0.35610,0.21962,2028,0.65765,0.34606,-0.01083,0.06254,0.65992,0.31601,269874.74220,...,19.2683,17.1794,12.5175,-5.2076,-3.1741,8.0271,1292.41919,42.75130,163.54240,94.7781
3,0.28306,0.16884,2227,0.57791,0.16854,-0.09972,0.14742,0.54604,0.16876,306729.77724,...,20.6949,17.3567,11.6246,-6.2461,-3.5825,6.9869,1097.58044,6.29760,385.99399,97.2635
4,0.58526,0.44648,1526,0.71589,0.65050,0.13226,0.12306,0.76043,0.65026,179272.43710,...,16.0769,15.9141,12.0073,-6.1475,-3.2039,8.9052,952.49194,344.36292,218.85001,84.4186


In [22]:
df_merged.columns

Index(['EVI_greenest_pixel_sent2', 'EVI_median_comp_sent2', 'FLATE_NR',
       'GNDVI_greenest_pixel_sent2', 'GNDVI_median_comp_sent2',
       'NDMI_greenest_pixel_sent2', 'NDMI_median_comp_sent2',
       'NDVI_greenest_pixel_sent2', 'NDVI_median_comp_sent2', 'POINT_X',
       'POINT_Y', 'SAVI_greenest_pixel_sent2', 'SAVI_median_comp_sent2',
       'layer', 'EVI_greenest_pixel_land7', 'EVI_median_comp_land7',
       'GNDVI_greenest_pixel_land7', 'GNDVI_median_comp_land7',
       'NDMI_greenest_pixel_land7', 'NDMI_median_comp_land7',
       'NDVI_greenest_pixel_land7', 'NDVI_median_comp_land7',
       'SAVI_greenest_pixel_land7', 'SAVI_median_comp_land7', 'ar50_artype',
       'ar50_skogbon', 'ar50_treslag', 'ar50_veg', 'corine_lc_2012',
       'geo_berggrunn', 'geo_grunnvann', 'geo_infiltr_evne', 'geo_losmasse',
       'geo_norge123', 'geology_norge', 'aspect', 'bioclim_1', 'bioclim_10',
       'bioclim_11', 'bioclim_12', 'bioclim_15', 'bioclim_17', 'bioclim_18',
       'bioclim_3', 'b

In [23]:
df_merged.shape

(22205, 80)

----
### 4. Prettify data frame

In [24]:
# Rename columns
df_merged.rename(columns={'POINT_X': 'x'}, inplace = True)
df_merged.rename(columns={'POINT_Y': 'y'}, inplace = True)
df_merged.rename(columns={'FLATE_NR': 'plot_id'}, inplace = True)
df_merged.rename(columns={'layer': 'vt'}, inplace = True)

In [25]:
df_merged.columns

Index(['EVI_greenest_pixel_sent2', 'EVI_median_comp_sent2', 'plot_id',
       'GNDVI_greenest_pixel_sent2', 'GNDVI_median_comp_sent2',
       'NDMI_greenest_pixel_sent2', 'NDMI_median_comp_sent2',
       'NDVI_greenest_pixel_sent2', 'NDVI_median_comp_sent2', 'x', 'y',
       'SAVI_greenest_pixel_sent2', 'SAVI_median_comp_sent2', 'vt',
       'EVI_greenest_pixel_land7', 'EVI_median_comp_land7',
       'GNDVI_greenest_pixel_land7', 'GNDVI_median_comp_land7',
       'NDMI_greenest_pixel_land7', 'NDMI_median_comp_land7',
       'NDVI_greenest_pixel_land7', 'NDVI_median_comp_land7',
       'SAVI_greenest_pixel_land7', 'SAVI_median_comp_land7', 'ar50_artype',
       'ar50_skogbon', 'ar50_treslag', 'ar50_veg', 'corine_lc_2012',
       'geo_berggrunn', 'geo_grunnvann', 'geo_infiltr_evne', 'geo_losmasse',
       'geo_norge123', 'geology_norge', 'aspect', 'bioclim_1', 'bioclim_10',
       'bioclim_11', 'bioclim_12', 'bioclim_15', 'bioclim_17', 'bioclim_18',
       'bioclim_3', 'bioclim_5', 'bioc

In [26]:
### Remove unwanted ending of VT strings
df_merged['vt'] = [x.replace("_p", "") for x in df_merged['vt']]
df_merged['vt']

0        1ab
1        1ab
2        1ab
3        1ab
4        1ab
        ... 
22200    8cd
22201    8cd
22202    8cd
22203     4c
22204     4c
Name: vt, Length: 22205, dtype: object

In [27]:
unsorted_columns = list(df_merged.columns.values)
first_columns = ["x", "y", "vt", "plot_id"]

In [28]:
# Remove columns to put first from full column list
for col in first_columns:
    unsorted_columns.remove(col)

# Then append the remaining columns after the ones to put first
for col in unsorted_columns:
    first_columns.append(col)

In [29]:
# Print to make sure desired resulted was achieved
print(first_columns)

['x', 'y', 'vt', 'plot_id', 'EVI_greenest_pixel_sent2', 'EVI_median_comp_sent2', 'GNDVI_greenest_pixel_sent2', 'GNDVI_median_comp_sent2', 'NDMI_greenest_pixel_sent2', 'NDMI_median_comp_sent2', 'NDVI_greenest_pixel_sent2', 'NDVI_median_comp_sent2', 'SAVI_greenest_pixel_sent2', 'SAVI_median_comp_sent2', 'EVI_greenest_pixel_land7', 'EVI_median_comp_land7', 'GNDVI_greenest_pixel_land7', 'GNDVI_median_comp_land7', 'NDMI_greenest_pixel_land7', 'NDMI_median_comp_land7', 'NDVI_greenest_pixel_land7', 'NDVI_median_comp_land7', 'SAVI_greenest_pixel_land7', 'SAVI_median_comp_land7', 'ar50_artype', 'ar50_skogbon', 'ar50_treslag', 'ar50_veg', 'corine_lc_2012', 'geo_berggrunn', 'geo_grunnvann', 'geo_infiltr_evne', 'geo_losmasse', 'geo_norge123', 'geology_norge', 'aspect', 'bioclim_1', 'bioclim_10', 'bioclim_11', 'bioclim_12', 'bioclim_15', 'bioclim_17', 'bioclim_18', 'bioclim_3', 'bioclim_5', 'bioclim_6', 'bioclim_7', 'bioclim_8', 'bioclim_9', 'dem100', 'growing_season_length', 'precip_5', 'precip_6'

In [30]:
# Create sorted list to create new data frame
sorted_df = df_merged[first_columns]

In [31]:
sorted_df.head()

,x,y,vt,plot_id,EVI_greenest_pixel_sent2,EVI_median_comp_sent2,GNDVI_greenest_pixel_sent2,GNDVI_median_comp_sent2,NDMI_greenest_pixel_sent2,NDMI_median_comp_sent2,...,tmax_6,tmax_8,tmax_9,tmin_5,tmin_9,topographic_wetness_index,total_insolation,valley_depth,vertical_distance_to_channel_network,visible_sky
0,270096.62361,6.948292e+06,1ab,2028,0.44837,0.31249,0.69518,0.58204,0.12542,0.01721,...,19.2190,17.1278,12.4009,-5.3641,-3.4312,8.9934,1320.79541,30.04170,176.19540,95.5727
1,270602.65752,6.948119e+06,1ab,2028,0.40884,0.21415,0.67526,0.46627,0.09299,-0.03565,...,19.2427,17.1531,12.4217,-5.2999,-3.3327,8.6541,1286.72034,9.76290,196.54530,97.7823
2,269874.74220,6.947897e+06,1ab,2028,0.35610,0.21962,0.65765,0.34606,-0.01083,0.06254,...,19.2683,17.1794,12.5175,-5.2076,-3.1741,8.0271,1292.41919,42.75130,163.54240,94.7781
3,306729.77724,6.930090e+06,1ab,2227,0.28306,0.16884,0.57791,0.16854,-0.09972,0.14742,...,20.6949,17.3567,11.6246,-6.2461,-3.5825,6.9869,1097.58044,6.29760,385.99399,97.2635
4,179272.43710,6.912279e+06,1ab,1526,0.58526,0.44648,0.71589,0.65050,0.13226,0.12306,...,16.0769,15.9141,12.0073,-6.1475,-3.2039,8.9052,952.49194,344.36292,218.85001,84.4186


In [32]:
sorted_df.shape

(22205, 80)

In [33]:
# No na values left?
sorted_df.isna().sum().sum()

0

#### Check VT occurrences, drop too rare types

In [34]:
merged_vts = sorted_df["vt"]

In [35]:
n_vts = merged_vts.value_counts()

In [36]:
n_vts

2ef     3004
9bc     2334
6a      1924
2c      1465
7b      1441
1ab     1425
4b      1222
8cd     1096
9ad      931
4a       894
4c       703
12b      680
3ab      617
12c      511
7c       487
7a       419
11b      391
8b       353
6b       348
2g       302
8a       259
1c       250
10ab     248
2d       167
2a       137
2b       126
10c      122
4e        96
5ab       95
4g        79
9e        47
12a       32
Name: vt, dtype: int64

In [37]:
# How many different VTS?
len(n_vts)

32

In [38]:
# Define min. number of occurrences, drop too rare ones
n_too_rare = 40
too_rare_index = n_vts[n_vts < n_too_rare].index

In [39]:
# Remove from DF
for rare_type in too_rare_index:
    sorted_df = sorted_df[sorted_df["vt"] != rare_type]

print(sorted_df.shape)

(22173, 80)


In [40]:
# Test again
merged_vts = sorted_df["vt"]
print(len(merged_vts.value_counts()))
merged_vts.value_counts()

31


2ef     3004
9bc     2334
6a      1924
2c      1465
7b      1441
1ab     1425
4b      1222
8cd     1096
9ad      931
4a       894
4c       703
12b      680
3ab      617
12c      511
7c       487
7a       419
11b      391
8b       353
6b       348
2g       302
8a       259
1c       250
10ab     248
2d       167
2a       137
2b       126
10c      122
4e        96
5ab       95
4g        79
9e        47
Name: vt, dtype: int64

----
### 5. Assign correct data types

In [41]:
pd.options.display.max_rows = None

In [42]:
len(sorted_df.columns)

80

In [43]:
sorted_df.dtypes

x                                       float64
y                                       float64
vt                                       object
plot_id                                   int64
EVI_greenest_pixel_sent2                float64
EVI_median_comp_sent2                   float64
GNDVI_greenest_pixel_sent2              float64
GNDVI_median_comp_sent2                 float64
NDMI_greenest_pixel_sent2               float64
NDMI_median_comp_sent2                  float64
NDVI_greenest_pixel_sent2               float64
NDVI_median_comp_sent2                  float64
SAVI_greenest_pixel_sent2               float64
SAVI_median_comp_sent2                  float64
EVI_greenest_pixel_land7                float64
EVI_median_comp_land7                   float64
GNDVI_greenest_pixel_land7              float64
GNDVI_median_comp_land7                 float64
NDMI_greenest_pixel_land7               float64
NDMI_median_comp_land7                  float64
NDVI_greenest_pixel_land7               

In [44]:
# Define variables to save as categoricals
cat_vars = (
    "vt",
    "ar50_artype",
    "ar50_skogbon",
    "ar50_treslag",
    "ar50_veg",
    "corine_lc_2012",
    "geo_berggrunn",
    "geo_grunnvann",
    "geo_infiltr_evne",
    "geo_losmasse",
    "geo_norge123",
    "geology_norge"    
)

<h3 style="color:red">Obs! Geo_norge123 and geology_norge are in fact identical</h3>
Remove from df.

In [45]:
sorted_df["geo_norge123"].equals(sorted_df["geology_norge"])

True

In [46]:
sorted_df = sorted_df.drop(columns=["geo_norge123"], axis=1)

In [47]:
# Loop through variables and convert to dtype
for var in cat_vars:
    
    if var in sorted_df.columns:
    
        # If int was read as float, change back before creating categories
        if sorted_df[var].dtype == 'float64':
            sorted_df[var] = sorted_df[var].astype(int)

        sorted_df[var] = sorted_df[var].astype('category')

In [48]:
cat_vars = [x for x in sorted_df.columns if (str(sorted_df.dtypes[x]) == 'category')]
sorted_df.loc[0:6, cat_vars]

,vt,ar50_artype,ar50_skogbon,ar50_treslag,ar50_veg,corine_lc_2012,geo_berggrunn,geo_grunnvann,geo_infiltr_evne,geo_losmasse,geology_norge
0,1ab,50,98,39,55,322,9,1,1,3,2
1,1ab,50,98,39,54,333,9,1,1,3,2
2,1ab,50,98,39,55,322,9,1,1,3,2
3,1ab,50,98,39,52,333,12,1,4,7,1
4,1ab,50,98,39,54,333,9,1,4,7,2
5,1ab,50,98,39,55,322,9,1,4,7,2
6,1ab,50,98,39,55,412,15,2,2,4,1


In [49]:
### Make sure there are no empty values
sorted_df.isnull().sum()

x                                       0
y                                       0
vt                                      0
plot_id                                 0
EVI_greenest_pixel_sent2                0
EVI_median_comp_sent2                   0
GNDVI_greenest_pixel_sent2              0
GNDVI_median_comp_sent2                 0
NDMI_greenest_pixel_sent2               0
NDMI_median_comp_sent2                  0
NDVI_greenest_pixel_sent2               0
NDVI_median_comp_sent2                  0
SAVI_greenest_pixel_sent2               0
SAVI_median_comp_sent2                  0
EVI_greenest_pixel_land7                0
EVI_median_comp_land7                   0
GNDVI_greenest_pixel_land7              0
GNDVI_median_comp_land7                 0
NDMI_greenest_pixel_land7               0
NDMI_median_comp_land7                  0
NDVI_greenest_pixel_land7               0
NDVI_median_comp_land7                  0
SAVI_greenest_pixel_land7               0
SAVI_median_comp_land7            

In [50]:
sorted_df.dtypes

x                                        float64
y                                        float64
vt                                      category
plot_id                                    int64
EVI_greenest_pixel_sent2                 float64
EVI_median_comp_sent2                    float64
GNDVI_greenest_pixel_sent2               float64
GNDVI_median_comp_sent2                  float64
NDMI_greenest_pixel_sent2                float64
NDMI_median_comp_sent2                   float64
NDVI_greenest_pixel_sent2                float64
NDVI_median_comp_sent2                   float64
SAVI_greenest_pixel_sent2                float64
SAVI_median_comp_sent2                   float64
EVI_greenest_pixel_land7                 float64
EVI_median_comp_land7                    float64
GNDVI_greenest_pixel_land7               float64
GNDVI_median_comp_land7                  float64
NDMI_greenest_pixel_land7                float64
NDMI_median_comp_land7                   float64
NDVI_greenest_pixel_

In [51]:
print(sorted_df.shape)

(22173, 79)


----
### 6. Save data

In [52]:
# Shuffle dataframe to ensure randomness
sorted_df = sorted_df.sample(frac=1, random_state=7).reset_index(drop=True)
sorted_df.head()

,x,y,vt,plot_id,EVI_greenest_pixel_sent2,EVI_median_comp_sent2,GNDVI_greenest_pixel_sent2,GNDVI_median_comp_sent2,NDMI_greenest_pixel_sent2,NDMI_median_comp_sent2,...,tmax_6,tmax_8,tmax_9,tmin_5,tmin_9,topographic_wetness_index,total_insolation,valley_depth,vertical_distance_to_channel_network,visible_sky
0,413948.16430,7.200240e+06,8b,2842,0.42104,0.40741,0.70197,0.66032,0.31809,0.14336,...,22.0491,22.0749,16.4459,-2.4543,-1.0046,8.2101,1073.48169,415.43869,7.57320,94.2645
1,180472.58945,6.821747e+06,2ef,1521,0.32259,0.21399,0.49835,0.35603,-0.12114,-0.10915,...,14.0074,15.6733,10.6097,-9.4873,-3.4765,6.2690,1098.24207,150.87869,317.84091,91.0748
2,323746.64336,6.947909e+06,2ef,2328,0.53046,0.40952,0.74365,0.64169,0.23448,0.05201,...,20.3088,18.4424,13.7082,-3.3195,-1.6668,9.0169,1252.09399,105.67400,38.37750,97.3012
3,827688.55243,7.758279e+06,2ef,5173,0.39231,0.35937,0.70110,0.65911,0.08476,0.01571,...,18.8220,18.6556,12.9306,-7.3210,-1.8053,13.6913,1003.45587,50.75290,55.44080,97.3278
4,180553.58548,6.605858e+06,6a,1509,0.51686,0.48097,0.73059,0.67669,0.35250,0.26084,...,22.8166,20.4597,16.0109,1.8245,2.3790,6.7679,1339.04956,298.20831,42.13300,94.7297


In [53]:
sorted_df.shape

(22173, 79)

In [54]:
sorted_df.to_csv("../data/interim/vtdata_full.csv", index=False)

In [55]:
# Also store as pickle to keep formatting
sorted_df.to_pickle("../data/interim/vtdata_full.pkl")

----
----
----